##### Libs

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

import sys

from my_functions import reduction_memory

import warnings
warnings.filterwarnings('ignore')

##### Read data

In [2]:
data = pd.read_csv('retail_train.csv')
data = reduction_memory(data)

data.head(5)

before:		230.09 MB
after:		141.41 MB
reduсed:	88.68 MB


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


##### Train test split

In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.shape[0], data_test.shape[0]

(2278490, 118314)

### Задание 0. Товар 999999
На вебинаре мы использовали товар 999999 - это товар, который купили пользователи, если они купиши товар из top5000. Используя этот товар мы смещяем качество рекомендаций. В какую сторону? Уберите этот товар и сравните с качеством на семинаре.

##### ответ:
Видно что при налицие товара 999999 качество сместилось в меньшую сторону. Скорее всего это произошло из-за того, что товар 999999 изначально не находится среди фактически купленных товаров, поэтому модели не могли его учитывать.

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(10)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8..."
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884..."
7,14,"[840601, 867293, 933067, 951590, 952408, 96569..."
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13..."
9,16,"[1062973, 1082185, 13007710]"


In [5]:
items = data_train['item_id'].value_counts()
items_logs = np.log(items)
items_weights = items_logs / items_logs.sum()

items_weights = items_weights.reset_index()
items_weights.columns = ['item_id', 'weights']

items_weights.head()

,item_id,weights
0,1082185,0.000078
1,6534178,0.000075
2,1029743,0.000073
3,995242,0.000072
4,1106523,0.000070


In [6]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    weights = np.array(items_weights['weights'])
    items = np.array(items_weights['item_id'])
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

Сделайте предсказания

In [7]:
%%time

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights))
result

Wall time: 5.17 s


,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1080989, 1119278, 1001646, 897859, 10356789]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1759896, 4249927, 1591286, 15971718, 2026840]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1488652, 1462685, 1159630, 7025171, 922608]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[8091615, 6391522, 858948, 1063023, 14111260]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[909769, 1221143, 935467, 2837912, 1104732]"
...,...,...,...
2037,2496,[6534178],"[1064686, 1070105, 13007239, 1080416, 877964]"
2038,2497,"[1016709, 9835695, 1132298, 16809501, 845294, ...","[327034, 968480, 1095227, 854130, 1033230]"
2039,2498,"[15716530, 834484, 901776, 914190, 958382, 972...","[1322314, 1076444, 1124577, 1194560, 1518675]"
2040,2499,"[867188, 877580, 902396, 914190, 951590, 95813...","[9654983, 1123454, 2422263, 1095001, 33944]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [8]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    
    recommended_list = np.array(recommended_list)
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [9]:
results = pd.read_pickle('predictions_basic.pcl')
results = pd.merge(result, results, on=('user_id'))
results = results.drop('actual_y', axis=1)

results.head(2)

,user_id,actual_x,weighted_random_recommendation,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1080989, 1119278, 1001646, 897859, 10356789]","[12384161, 1090291, 821787, 8157372, 10311412]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1759896, 4249927, 1591286, 15971718, 2026840]","[435678, 5574265, 1000824, 10285141, 12456791]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [10]:
def evaluete_pred(data, true):
    algs = {}
    for col in data.columns[2:]:
        res = data.apply(lambda row: precision_at_k(row[col], row[true]), axis=1).mean()
        algs[col] = res
        print(f'{col} : {res}')

In [11]:
evaluete_pred(results, 'actual_x')

weighted_random_recommendation : 0.0004897159647404506
random_recommendation : 0.0005876591576885406
popular_recommendation : 0.15523996082272082
itemitem : 0.13692458374142857
cosine : 0.13290891283055686
tfidf : 0.1389813907933383
own_purchases : 0.17969311132876015


In [12]:
for item in results['itemitem']:
    item.remove(999999)

evaluete_pred(results, 'actual_x')

weighted_random_recommendation : 0.0004897159647404506
random_recommendation : 0.0005876591576885406
popular_recommendation : 0.15523996082272082
itemitem : 0.17115572967678747
cosine : 0.13290891283055686
tfidf : 0.1389813907933383
own_purchases : 0.17969311132876015


##### ответ:

own_purchases - показывает лучший результат (0.17969311132876015)

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров

In [13]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [14]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [15]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()
top_5000[:5]

[6534178, 6533889, 6534166, 6544236, 1404121]

In [16]:
# выборка купившая товары из топ 5000
data_test_top_5k = data_test[data_test['item_id'].isin(top_5000)]
result1 = data_test_top_5k.groupby('user_id')['item_id'].unique().reset_index()
result1.columns=['user_id', 'actual']
result1.head()

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 914190, 95804..."
1,3,"[851057, 872021, 878302, 879948, 909638, 91320..."
2,6,"[920308, 926804, 1017061, 1078346, 1120741, 82..."
3,7,"[840386, 889774, 898068, 909714, 953476, 97699..."
4,8,"[835098, 872137, 910439, 924610, 1041259, 5569..."


In [17]:
%%time

items = data_train.item_id.unique()

result1['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result1.head()

Wall time: 4.71 s


,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 914190, 95804...","[1024189, 1088462, 886370, 13039019, 1061604]"
1,3,"[851057, 872021, 878302, 879948, 909638, 91320...","[7465730, 1074773, 981900, 819054, 1130070]"
2,6,"[920308, 926804, 1017061, 1078346, 1120741, 82...","[9305129, 1847089, 859203, 9831544, 1015823]"
3,7,"[840386, 889774, 898068, 909714, 953476, 97699...","[1049897, 1026504, 5567874, 1107251, 844173]"
4,8,"[835098, 872137, 910439, 924610, 1041259, 5569...","[9527306, 9221170, 824521, 9654828, 957537]"


In [18]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result1['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result1.head()

Wall time: 172 ms


,user_id,actual,random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 914190, 95804...","[1024189, 1088462, 886370, 13039019, 1061604]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[851057, 872021, 878302, 879948, 909638, 91320...","[7465730, 1074773, 981900, 819054, 1130070]","[6534178, 6533889, 1029743, 6534166, 1082185]"
2,6,"[920308, 926804, 1017061, 1078346, 1120741, 82...","[9305129, 1847089, 859203, 9831544, 1015823]","[6534178, 6533889, 1029743, 6534166, 1082185]"
3,7,"[840386, 889774, 898068, 909714, 953476, 97699...","[1049897, 1026504, 5567874, 1107251, 844173]","[6534178, 6533889, 1029743, 6534166, 1082185]"
4,8,"[835098, 872137, 910439, 924610, 1041259, 5569...","[9527306, 9221170, 824521, 9654828, 957537]","[6534178, 6533889, 1029743, 6534166, 1082185]"


In [19]:
%%time

result1['weighted_random_recommendation'] = result1['user_id'].apply(lambda x: weighted_random_recommendation(items_weights))
result1.head()

Wall time: 4.96 s


,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 914190, 95804...","[1024189, 1088462, 886370, 13039019, 1061604]","[6534178, 6533889, 1029743, 6534166, 1082185]","[15716685, 1770400, 15716286, 5707620, 14024505]"
1,3,"[851057, 872021, 878302, 879948, 909638, 91320...","[7465730, 1074773, 981900, 819054, 1130070]","[6534178, 6533889, 1029743, 6534166, 1082185]","[842330, 910656, 13416730, 1101104, 13004478]"
2,6,"[920308, 926804, 1017061, 1078346, 1120741, 82...","[9305129, 1847089, 859203, 9831544, 1015823]","[6534178, 6533889, 1029743, 6534166, 1082185]","[9368398, 938628, 866828, 973305, 8293418]"
3,7,"[840386, 889774, 898068, 909714, 953476, 97699...","[1049897, 1026504, 5567874, 1107251, 844173]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1828801, 9446847, 1060926, 1067667, 1987571]"
4,8,"[835098, 872137, 910439, 924610, 1041259, 5569...","[9527306, 9221170, 824521, 9654828, 957537]","[6534178, 6533889, 1029743, 6534166, 1082185]","[373795, 1105436, 10198392, 13213082, 6547687]"


In [20]:
evaluete_pred(result1, 'actual')

random_recommendation : 0.00030395136778115504
popular_recommendation : 0.1605876393110415
weighted_random_recommendation : 0.00060790273556231


- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [21]:
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0)

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [23]:
%%time

model = ItemItemRecommender(K=2, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)


Wall time: 1.13 s


In [24]:
%%time

results['itemitem'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

Wall time: 68.7 ms


In [25]:
results.apply(lambda row: precision_at_k(row['itemitem'], row['actual_x']), axis=1).mean()

0.1920094678419821

##### Ответ:
Лучший показатель для itemitem при K = 2

- Попробуйте стратегии ансамблирования изученных алгоритмов

-

### Задание 4. Улучшение детерминированных алгоритмов
На семинаре мы рассматривали 

Далее $U \equiv N_i(u) $

$$r_{u,i} =  \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)r_{v, i}$$
$$ S = \sum\limits_{v \in U} \operatorname{sim}(u,v)$$

Предлагается улучшить эту формулу и учесть средние предпочтения всех пользователей

$$r_{u,i} = \mu + \bar{r_u} + \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)(r_{v, i}-\bar{r_{v}} - \mu)$$

Какие смысл имееют $ \mu $ и $ \bar{r_u}$ ?

##### ответ:

$\mu$ - средняя оценка по всей таблице<br>
$\bar{r_u}$ - средний рейтинг товаров, которые оценил пользователь
$sim$ - функция схожести, (1 - расстояние)<br>
$u$ - пользователь<br>
$v$ - пользователь из множества $N_i(u)$<br>
$i$ - товар<br>
$r_{u,i}$ - оценка которая мы предсказываем паре<br>
$N_i(u)$ - массив пользователей N которые оценили товар i, который оценил пользователь u

Реализуйте алгоритм, прогнозирующий рейтинги на основе данной формулы, на numpy (векторизованно!)

В качестве схожести возьмите CosineSimilarity.

Примените к user_item_matrix. В качестве рейтингов возьмите количество или стоимость купленного товара. 
Данный алгоритм предсказывает рейтинги. Как на основании предсказанных рейтингов предсказать факт покупки?

Предложите вариант.
Посчитайте accuracy@5 и сравните с алгоритмами, разобранными на вебинаре.

In [26]:
from sklearn.metrics.pairwise import cosine_similarity

In [27]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0)

In [30]:
mu = user_item_matrix.mean().mean()
ru_overline = user_item_matrix.mean(axis=1)

$(r_{v, i}-\bar{r_{v}} - \mu)$

In [57]:
(user_item_matrix.T - ru_overline.values - mu).T

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,-0.483655,-0.483655,-0.483655,-0.483655,-0.483655,-0.483655,-0.483655,-0.483655,-0.483655,-0.483655,...,-0.483655,-0.483655,-0.483655,-0.483655,0.516345,-0.483655,-0.483655,-0.483655,-0.483655,-0.483655
2,-0.313289,-0.313289,-0.313289,-0.313289,-0.313289,-0.313289,-0.313289,-0.313289,-0.313289,-0.313289,...,-0.313289,-0.313289,-0.313289,-0.313289,-0.313289,-0.313289,-0.313289,-0.313289,-0.313289,-0.313289
3,-0.356281,-0.356281,-0.356281,-0.356281,-0.356281,-0.356281,-0.356281,-0.356281,-0.356281,-0.356281,...,-0.356281,-0.356281,-0.356281,-0.356281,-0.356281,-0.356281,-0.356281,-0.356281,-0.356281,-0.356281
4,-0.242504,-0.242504,-0.242504,-0.242504,-0.242504,-0.242504,-0.242504,-0.242504,-0.242504,-0.242504,...,-0.242504,-0.242504,-0.242504,-0.242504,-0.242504,-0.242504,-0.242504,-0.242504,-0.242504,-0.242504
5,-0.226107,-0.226107,-0.226107,-0.226107,-0.226107,-0.226107,-0.226107,-0.226107,-0.226107,-0.226107,...,-0.226107,-0.226107,-0.226107,-0.226107,-0.226107,-0.226107,-0.226107,-0.226107,-0.226107,-0.226107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2496,-0.466059,-0.466059,-0.466059,-0.466059,-0.466059,-0.466059,-0.466059,-0.466059,-0.466059,-0.466059,...,-0.466059,-0.466059,-0.466059,-0.466059,-0.466059,-0.466059,-0.466059,-0.466059,-0.466059,-0.466059
2497,-0.537445,-0.537445,-0.537445,-0.537445,-0.537445,-0.537445,-0.537445,-0.537445,-0.537445,-0.537445,...,-0.537445,0.462555,-0.537445,-0.537445,-0.537445,-0.537445,-0.537445,-0.537445,-0.537445,-0.537445
2498,1.659116,-0.340884,-0.340884,-0.340884,-0.340884,-0.340884,-0.340884,-0.340884,-0.340884,-0.340884,...,-0.340884,-0.340884,-0.340884,-0.340884,-0.340884,-0.340884,-0.340884,-0.340884,-0.340884,-0.340884
